In [ ]:
# Our strategy rules are:

# Buy Signal: If the stock's 50-day Simple Moving Average (SMA) crosses above its 200-day SMA, AND its Relative Strength Index (RSI) is below 70 (not overbought).
# Sell Signal: If the stock's 50-day SMA crosses below its 200-day SMA, AND its RSI is above 30 (not oversold).
# Hold Signal: If neither a buy nor a sell signal is present.

In [10]:
import pandas as pd
import numpy as np 
import yfinance as yf 

In [43]:

# try:
#     aapl_data = yf.download('AAPL', start='2020-01-01', end='2025-01-01')
#     print(aapl_data)
#     # Fix for multi-index columns
#     if isinstance(aapl_data.columns, pd.MultiIndex):
#         aapl_data = aapl_data['Close']['AAPL'].copy().to_frame()
#     else:
#         aapl_data = aapl_data[['Close']].copy()

#     print("AAPL data downloaded successfully!")
#     print(aapl_data.head())

# except Exception as e:
#     print(f"Could not download data from yfinance. Using dummy data. Error: {e}")
    
#     dates = pd.date_range(start='2020-01-01', periods=2000, freq='D')
#     aapl_data = pd.DataFrame({
#         'Close': np.random.rand(2000) * 100 + 50
#     }, index=dates)

#     print(aapl_data.head())

import yfinance as yf
import pandas as pd
import numpy as np

try:
    # Step 1: Download
    aapl_data = yf.download('AAPL', start='2020-01-01', end='2025-01-01', group_by='ticker', progress=False)

    # Step 2: Check if it's empty
    if aapl_data.empty or 'Close' not in aapl_data.columns.get_level_values(0):
        raise ValueError("Downloaded data is empty or not in expected format.")

    # Step 3: Fix MultiIndex
    if isinstance(aapl_data.columns, pd.MultiIndex):
        aapl_data = aapl_data['Close']['AAPL'].copy().to_frame(name='Close')
    else:
        aapl_data = aapl_data[['Close']].copy()

    print("✅ AAPL data downloaded successfully!")
    print(aapl_data.head())

except Exception as e:
    print(f"⚠️ Could not download data from yfinance. Using dummy data. Error: {e}")

    # Step 4: Dummy data fallback
    dates = pd.date_range(start='2020-01-01', periods=2000, freq='D')
    aapl_data = pd.DataFrame({
        'Close': np.random.rand(2000) * 100 + 50
    }, index=dates)

    print("📊 Dummy data preview:")
    print(aapl_data)




1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


⚠️ Could not download data from yfinance. Using dummy data. Error: Downloaded data is empty or not in expected format.
📊 Dummy data preview:
                 Close
2020-01-01   65.883876
2020-01-02   96.555322
2020-01-03   51.082544
2020-01-04   71.895462
2020-01-05   73.844515
...                ...
2025-06-18  100.810536
2025-06-19  109.615668
2025-06-20   51.267612
2025-06-21   91.167746
2025-06-22  101.511523

[2000 rows x 1 columns]


In [ ]:
##  Functions: Calculating SMAs and RSI (Modularizing our code)

In [45]:
def calculate_sma(data_series, window):
    return data_series.rolling(window = window).mean()

In [47]:
def calculate_rsi(data_series, window = 14):
    delta = data_series.diff()
    gain = (delta.where(delta>0,0)).rolling(window=window).mean()
    loss= (-delta.where(delta<0,0)).rolling(window=window).mean()
    rs = gain/loss
    rsi = 100-(100/(1+rs))
    return rsi

In [49]:
# apply function to our aapl data

aapl_data['SMA_50'] = calculate_sma(aapl_data['Close'], 50)
aapl_data['SMA_200'] = calculate_sma(aapl_data['Close'], 200)
aapl_data['RSI'] = calculate_rsi(aapl_data['Close'], 14)
# drop initial NaN values created by rolling window calculations
aapl_data.dropna(inplace=True)
print(aapl_data.head())

                 Close      SMA_50     SMA_200        RSI
2020-07-18  105.038247  102.145906   99.450361  51.694077
2020-07-19   78.963124  100.809696   99.515757  48.083477
2020-07-20   63.439660  100.746891   99.350179  44.152097
2020-07-21  141.860979  101.231161   99.804071  57.084542
2020-07-22  125.889166  101.956726  100.074040  48.695525


In [53]:
# Variables and datatype 
initial_cash: int = 100000
shares_held: int  = 0

In [55]:
# float (float): 
transaction_cost_per_share: float = 0.01 
current_cash : float = float(initial_cash)
portfolio_value: float = 0.0


In [ ]:
# boolean (bool):

sma_50_above_200_prev: bool = False
sma_50_below_200_prev: bool = False

In [59]:
# Strings

stock_ticker: str = "AAPL"
strategy_name: str = " SMA CROSSOVER WITH RSI FILTER"

In [1]:
#List of technical indicators 

indicator_list = ['sma_50','sma_200','rsi','macd']
tracked_stocks = ['aapl','msft','googl']


In [3]:
# tuple 



In [13]:
# Dictionary to store configuration for different trading strategies 

# Dictionary to store configuration for different trading strategies
strategy_configs = {
    "SMACrossover": {"sma_short": 50, "sma_long": 200, "rsi_filter": True},
    "Momentum": {"lookback": 10, "threshold": 0.02}
}

# Storing XAI explanation results for a specific trade
trade_explanation = {
    "date": "2023-10-26",
    "action": "BUY",
    "reason_summary": "SMA crossover with strong upward momentum and low RSI.",
    "feature_contributions": {
        "SMA_50": 0.4,
        "SMA_200": 0.3,
        "RSI": 0.2,
        "Volume": 0.1
    },
    "counterfactual_example": {
        "if_rsi_was_80": "would NOT have bought"
    }
}

print(f"Reasons for trade on {trade_explanation['date']}: {trade_explanation['reason_summary']}")
print(f"Top contributor: {max(trade_explanation['feature_contributions'], key=trade_explanation['feature_contributions'].get)}")

print(trade_explanation['feature_contributions'], trade_explanation['feature_contributions'].get)


Reasons for trade on 2023-10-26: SMA crossover with strong upward momentum and low RSI.
Top contributor: SMA_50
{'SMA_50': 0.4, 'SMA_200': 0.3, 'RSI': 0.2, 'Volume': 0.1} <built-in method get of dict object at 0x000001AD50E997C0>
